<a href="https://colab.research.google.com/github/mateuschh/mobilenetv2-laranjas/blob/main/ssd_mobilenetv2_embrapa_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'14 a 16.gsheet'        Dissertação	   Pré-Wedding
'Arquivos anteriores'  'Fotos Casamento'   ssd_mobilenetv2_embrapa
 Classroom	       'Meet Recordings'   ssd_mobilenetv2_embrapa_evaluation
'Colab Notebooks'      'My Drive'	  'Treinos anteriores'
 Datasets	        Object_Detection


In [ ]:
pip install -U numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 57.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.2 which is incompatible.


In [ ]:
%%bash
git clone https://github.com/tensorflow/models.git

cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

python object_detection/builders/model_builder_tf2_test.py

from __future__ import division, print_function, absolute_import

Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing meta

Cloning into 'models'...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flax 0.7.2 requires PyYAML>=5.4.1, but you have pyyaml 5.3.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
pydantic 2.2.1 requires typing-extensions>=4.6.1, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.6.1 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
2023-08-29 13:51:02.786098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 13:51:04.091112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:3

CalledProcessError: ignored

In [ ]:
import pandas as pd
import numpy as np
import os
import io
import tensorflow as tf
import csv
import re
import cv2
import glob
import xml.etree.ElementTree as ET
import matplotlib
import matplotlib.pyplot as plt
import pathlib
import scipy.misc

from six import BytesIO

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import dataset_util

from PIL import Image, ImageDraw, ImageFont
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

%matplotlib inline

In [ ]:
# Test each model with some checkpoints
pl_config = '/mydrive/Object_Detection/ssd/models/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
ckpt_dir = '/mydrive/Object_Detection/ssd/models/research/object_detection/configs/tf2/training'
#ckpt_dir = '/mydrive/Object_Detection/ssd/models/research/object_detection/configs/tf2/trainingv2'
model_dir_config = '/mydrive/Object_Detection/ssd/models/research/object_detection/configs/tf2/training'

!python /mydrive/Object_Detection/ssd/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pl_config} \
    --model_dir={model_dir_config} \
    --checkpoint_dir={ckpt_dir}

2023-08-18 12:49:47.472324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
W0818 12:49:53.131638 132149114085376 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0818 12:49:53.131961 132149114085376 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0818 12:49:53.132081 132149114085376 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0818 12:49:53.132190 132149114085376 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0818 12:49:53.132374 132149114085376 model_lib_v2.py:1106] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:Reading unweighted datasets: ['/mydrive/Object_Detection/ssd/da

In [ ]:
!kill 13033

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir '/mydrive/Object_Detection/ssd/models/research/object_detection/configs/tf2/training'

<IPython.core.display.Javascript object>